# Convert 3-stage gridsearch log → CSV + Excel leaderboards

This notebook parses `phase_results.txt` and exports:

- `results/phase_results_parsed.csv` — tidy table with one row per `(phase, run_idx, epoch)`.
- `results/phase_results_parsed.xlsx` — Excel workbook with:
  - `parsed_epochs`
  - `leaderboard_phase1`, `leaderboard_phase2`, `leaderboard_phase3`

**Differences vs the BaseTrain parser**
- Adds a `phase` column (1/2/3) + `phase_name` (e.g., baseline, grid search).
- Generates a separate leaderboard **per phase**.
- Uses a single seed: `38042`.


In [1]:
from pathlib import Path

# =============================================================================
# CONFIGURATION
# =============================================================================

# Global variable: relative directory where outputs will be written.
CSV_REL_DIR = "../Structured Outputs/3-Stage/"

# The log was run over 5 seeds in this order (not printed in the log itself).
SEED = [38042]

# Input log file path (relative to repo root)
INPUT_TXT_PATH = Path("../Raw Outputs/3-Stage/phase_results.txt")

# Output filenames (written inside CSV_REL_DIR)
OUTPUT_CSV_NAME = "3_stage.csv"
OUTPUT_XLSX_NAME = "3_stage.xlsx"

# Derived paths
OUTPUT_DIR = Path(CSV_REL_DIR)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_CSV_PATH = OUTPUT_DIR / OUTPUT_CSV_NAME
OUTPUT_XLSX_PATH = OUTPUT_DIR / OUTPUT_XLSX_NAME

print("Will read:", INPUT_TXT_PATH)
print("Will write CSV:", OUTPUT_CSV_PATH)
print("Will write XLSX:", OUTPUT_XLSX_PATH)
print("Seeds:", SEED)


Will read: ../Raw Outputs/3-Stage/phase_results.txt
Will write CSV: ../Structured Outputs/3-Stage/3_stage.csv
Will write XLSX: ../Structured Outputs/3-Stage/3_stage.xlsx
Seeds: [38042]


## 1) Regex + parsing helpers

We parse:
- Phase summary lines like: `Phase 1 (baseline): total runs=27, epochs per run=20`
- Run header lines like: `Phase 1 (baseline) run 3/27 | lr=... wd=...`
- Epoch lines like: `epoch 001/20 | loss ... | train_acc ...% | test_acc ...%`

Normalization:
- values ending with `%` become floats in `*_pct` columns.


In [2]:
import re

SUMMARY_RE = re.compile(
    r"^Phase\s+(?P<phase>\d+)\s+\((?P<name>[^)]+)\):\s*total runs=(?P<runs>\d+),\s*epochs per run=(?P<epochs>\d+)\s*$",
    re.IGNORECASE,
)

RUN_HEADER_RE = re.compile(
    r"^Phase\s+(?P<phase>\d+)\s+\((?P<name>[^)]+)\)\s+run\s+(?P<run_idx>\d+)\s*/\s*(?P<run_total>\d+)\s*\|\s*(?P<hparams>.+?)\s*$",
    re.IGNORECASE,
)

EPOCH_RE = re.compile(
    r"^\s*epoch\s+(?P<epoch>\d+)\s*/\s*(?P<epoch_total>\d+)\s*\|\s*(?P<rest>.+?)\s*$",
    re.IGNORECASE,
)

def parse_hparams(hparams_str: str) -> dict:
    """
    Parse: 'lr=0.0001 wd=0.02 bs=256 drop_path_rate=0.0' -> dict with numeric types.
    """
    out = {}
    for token in hparams_str.strip().split():
        if "=" not in token:
            continue
        k, v = token.split("=", 1)
        if re.fullmatch(r"-?\d+", v):
            out[k] = int(v)
        else:
            try:
                out[k] = float(v)
            except ValueError:
                out[k] = v
    return out

def parse_epoch_kv(rest: str) -> dict:
    """
    Parse the right side of an epoch line: 'key value | key value | ...'
    Normalize percent values -> *_pct.
    """
    kv = {}
    parts = [p.strip() for p in rest.split("|")]
    for p in parts:
        if not p or " " not in p:
            continue
        key, val = p.split(" ", 1)
        val = val.strip()

        if val.endswith("%"):
            try:
                kv[f"{key}_pct"] = float(val[:-1])
            except ValueError:
                kv[f"{key}_pct"] = None
        else:
            try:
                kv[key] = float(val)
            except ValueError:
                kv[key] = val
    return kv


## 2) Parse into a DataFrame and write CSV

We keep track of:
- `phase`, `phase_name`
- `phase_total_runs`, `phase_epochs_per_run` (from summary lines)
- per-run hyperparameters (`lr`, `wd`, `bs`, `drop_path_rate`)

We also add `seed=SEED` for every row.

If any epoch rows appear before a run header (rare), we **fill missing hyperparameters**
within `(phase, run_idx)` using the first non-null value observed for that run.


In [3]:
import pandas as pd

current = {
    "seed": SEED,
    "phase": None,
    "phase_name": None,
    "phase_total_runs": None,
    "phase_epochs_per_run": None,
    "run_idx": None,
    "run_total": None,
    "lr": None,
    "wd": None,
    "bs": None,
    "drop_path_rate": None,
}

records = []

with INPUT_TXT_PATH.open("r", encoding="utf-8", errors="replace") as f:
    for line in f:
        line = line.rstrip("\n")

        m = SUMMARY_RE.match(line)
        if m:
            current["phase"] = int(m.group("phase"))
            current["phase_name"] = m.group("name").strip()
            current["phase_total_runs"] = int(m.group("runs"))
            current["phase_epochs_per_run"] = int(m.group("epochs"))
            continue

        m = RUN_HEADER_RE.match(line)
        if m:
            current["phase"] = int(m.group("phase"))
            current["phase_name"] = m.group("name").strip()
            current["run_idx"] = int(m.group("run_idx"))
            current["run_total"] = int(m.group("run_total"))
            current.update(parse_hparams(m.group("hparams")))
            continue

        m = EPOCH_RE.match(line)
        if m:
            rec = dict(current)
            rec["epoch"] = int(m.group("epoch"))
            rec["epoch_total"] = int(m.group("epoch_total"))
            rec.update(parse_epoch_kv(m.group("rest")))
            records.append(rec)

df = pd.DataFrame.from_records(records)

# Fill missing hyperparameters (only fills NaNs; never overwrites)
HYPERPARAM_COLS = ["lr", "wd", "bs", "drop_path_rate"]
for col in HYPERPARAM_COLS:
    if col in df.columns:
        df[col] = df.groupby(["phase", "run_idx"])[col].transform(
            lambda s: s.fillna(s.dropna().iloc[0] if s.notna().any() else pd.NA)
        )

# Column ordering
preferred_cols = [
    "seed",
    "phase", "phase_name", "phase_total_runs", "phase_epochs_per_run",
    "run_idx", "run_total", "lr", "wd", "bs", "drop_path_rate",
    "epoch", "epoch_total",
    "loss", "train_acc_pct", "test_acc_pct",
]
cols = preferred_cols + [c for c in df.columns if c not in preferred_cols]
df = df[cols].sort_values(["phase", "run_idx", "epoch"]).reset_index(drop=True)

df.to_csv(OUTPUT_CSV_PATH, index=False)

print("Rows:", len(df))
print("Phases:", sorted(df["phase"].dropna().unique().tolist()) if len(df) else [])
print("Wrote:", OUTPUT_CSV_PATH.resolve())
df.head()


Rows: 1890
Phases: [1, 2, 3]
Wrote: /Users/etaashpatel/Documents/Final Project/Structured Outputs/3-Stage/3_stage.csv


,seed,phase,phase_name,phase_total_runs,phase_epochs_per_run,run_idx,run_total,lr,wd,bs,drop_path_rate,epoch,epoch_total,loss,train_acc_pct,test_acc_pct
0,[38042],1,baseline,27,20,1,27,0.0001,0.02,256,0.0,1,20,2.0423,23.86,28.48
1,[38042],1,baseline,27,20,1,27,0.0001,0.02,256,0.0,2,20,1.8476,31.49,34.58
2,[38042],1,baseline,27,20,1,27,0.0001,0.02,256,0.0,3,20,1.7396,35.84,36.33
3,[38042],1,baseline,27,20,1,27,0.0001,0.02,256,0.0,4,20,1.6291,40.26,43.45
4,[38042],1,baseline,27,20,1,27,0.0001,0.02,256,0.0,5,20,1.4833,45.89,47.58


## 3) Export Excel workbook with one leaderboard per phase

For each phase:
- one row per `run_idx`
- ranked by **best** `test_acc_pct` achieved over epochs
- also includes the final-epoch metrics for sanity checking


In [4]:
import pandas as pd

# Ensure numeric for ranking/aggregation
for c in ["test_acc_pct", "train_acc_pct", "loss"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

def build_leaderboard(df_phase: pd.DataFrame) -> pd.DataFrame:
    best_rows = (
        df_phase.dropna(subset=["test_acc_pct"])
                .sort_values(["run_idx", "test_acc_pct", "epoch"], ascending=[True, False, True])
                .groupby("run_idx", as_index=False)
                .head(1)
                .set_index("run_idx")
    )
    final_rows = (
        df_phase.sort_values(["run_idx", "epoch"])
                .groupby("run_idx", as_index=False)
                .tail(1)
                .set_index("run_idx")
    )

    lb = pd.DataFrame({
        "run_idx": best_rows.index,
        "seed": best_rows.get("seed"),
        "phase": best_rows.get("phase"),
        "phase_name": best_rows.get("phase_name"),
        "lr": best_rows.get("lr"),
        "wd": best_rows.get("wd"),
        "bs": best_rows.get("bs"),
        "drop_path_rate": best_rows.get("drop_path_rate"),
        "best_test_acc_pct": best_rows["test_acc_pct"],
        "best_epoch": best_rows["epoch"],
        "best_loss": best_rows.get("loss"),
        "final_test_acc_pct": final_rows.get("test_acc_pct"),
        "final_loss": final_rows.get("loss"),
    }).reset_index(drop=True)

    # Rank by best test accuracy (tie-break by best_loss if available)
    sort_cols = ["best_test_acc_pct"]
    asc = [False]
    if "best_loss" in lb.columns:
        sort_cols.append("best_loss")
        asc.append(True)

    lb = lb.sort_values(sort_cols, ascending=asc).reset_index(drop=True)
    lb.insert(0, "rank", lb.index + 1)
    return lb

phase_values = sorted([int(p) for p in df["phase"].dropna().unique().tolist()])

with pd.ExcelWriter(OUTPUT_XLSX_PATH, engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name="parsed_epochs", index=False)
    for p in phase_values:
        lb = build_leaderboard(df[df["phase"] == p].copy())
        lb.to_excel(writer, sheet_name=f"leaderboard_phase{p}"[:31], index=False)

print("Wrote:", OUTPUT_XLSX_PATH.resolve())
print("Sheets:", ["parsed_epochs"] + [f"leaderboard_phase{p}" for p in phase_values])

# Preview top 10 of phase 1 leaderboard
if phase_values:
    display(build_leaderboard(df[df["phase"] == phase_values[0]].copy()).head(10))


Wrote: /Users/etaashpatel/Documents/Final Project/Structured Outputs/3-Stage/3_stage.xlsx
Sheets: ['parsed_epochs', 'leaderboard_phase1', 'leaderboard_phase2', 'leaderboard_phase3']


,rank,run_idx,seed,phase,phase_name,lr,wd,bs,drop_path_rate,best_test_acc_pct,best_epoch,best_loss,final_test_acc_pct,final_loss
0,1,19,[38042],1,baseline,0.00100,0.02,256,0.0,78.57,20,0.5172,78.57,0.5172
1,2,22,[38042],1,baseline,0.00100,0.07,256,0.0,78.25,19,0.5489,78.16,0.5351
2,3,20,[38042],1,baseline,0.00100,0.02,256,0.1,78.03,20,0.5637,78.03,0.5637
3,4,25,[38042],1,baseline,0.00100,0.15,256,0.0,77.68,20,0.5533,77.68,0.5533
4,5,24,[38042],1,baseline,0.00100,0.07,256,0.2,77.38,20,0.6051,77.38,0.6051
5,6,27,[38042],1,baseline,0.00100,0.15,256,0.2,77.22,19,0.6284,77.22,0.6153
6,7,23,[38042],1,baseline,0.00100,0.07,256,0.1,76.74,19,0.6105,76.59,0.5956
7,8,26,[38042],1,baseline,0.00100,0.15,256,0.1,76.58,20,0.6212,76.58,0.6212
8,9,21,[38042],1,baseline,0.00100,0.02,256,0.2,76.38,20,0.6279,76.38,0.6279
9,10,16,[38042],1,baseline,0.00033,0.15,256,0.0,76.25,20,0.5950,76.25,0.5950
